In [1]:
!pip freeze | grep scikit-learn

scikit-learn==1.7.0


In [2]:
!python -V

Python 3.12.1


In [3]:
import pickle
import pandas as pd

In [4]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

/home/codespace/venv/lib/python3.12/site-packages/sklearn/base.py:440: InconsistentVersionWarning: Trying to unpickle estimator DictVectorizer from version 1.5.0 when using version 1.7.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/codespace/venv/lib/python3.12/site-packages/sklearn/base.py:440: InconsistentVersionWarning: Trying to unpickle estimator LinearRegression from version 1.5.0 when using version 1.7.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [5]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [8]:
year = 2023
month = 3
df = read_data(f'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_{year}-{month:02d}.parquet')

In [9]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [15]:
# Q1: duration stdev
print(pd.DataFrame(y_pred).std())

0    6.24749
dtype: float64


In [22]:
# Q2
df['ride_id'] = f'{year:04d}/{month:02d}_' + df.index.astype('str')

import os
os.makedirs("out", exist_ok=True)
output_file = f"out/predicted_yellow_{year}-{month:02d}.parquet"

df_result = pd.DataFrame()
df_result['ride_id'] = df['ride_id']
df_result['predicted_duration'] = y_pred
df_result.to_parquet(
    output_file,
    engine='pyarrow',
    compression=None,
    index=False
)
print(df_result)

                 ride_id  predicted_duration
0              2023/03_0           16.245906
1              2023/03_1           26.134796
2              2023/03_2           11.884264
3              2023/03_3           11.997720
4              2023/03_4           10.234486
...                  ...                 ...
3403761  2023/03_3403761           11.952635
3403762  2023/03_3403762           20.049958
3403763  2023/03_3403763           11.595336
3403764  2023/03_3403764           13.113178
3403765  2023/03_3403765           12.899992

[3316216 rows x 2 columns]


In [24]:
!wc --bytes out/predicted_yellow*

68641024 out/predicted_yellow_2023-03.parquet


In [26]:
# Q3
!jupyter nbconvert --to script starter.ipynb

[NbConvertApp] Converting notebook starter.ipynb to script
[NbConvertApp] Writing 1619 bytes to starter.py


In [1]:
# Q4
!pip freeze | grep -e pickle -e pandas -e os -e scikit-learn -e numpy

aiosqlite==0.21.0
cloudpickle==3.1.1
numpy==2.3.1
pandas==2.3.0
python-dateutil==2.9.0.post0
scikit-learn==1.5.0
xgboost==3.0.2


In [34]:
!PIPENV_IGNORE_VIRTUALENVS=1 pipenv install --python 3.10 scikit-learn==1.5.0 pandas==2.3.0 numpy pyarrow

Creating a Pipfile for this project...
Pipfile.lock not found, creating...
Locking  dependencies...
Locking  dependencies...
Updated Pipfile.lock 
(702ad05de9bc9de99a4807c8dde1686f31e0041d7b5f6f6b74861195a52110f5)!
To activate this project's virtualenv, run pipenv shell.
Alternatively, run a command inside the virtualenv with pipenv run.
Installing scikit-learn==1.5.0...
✔ Installation Succeeded
Installing pandas==2.3.0...
✔ Installation Succeeded
To activate this project's virtualenv, run pipenv shell.
Alternatively, run a command inside the virtualenv with pipenv run.
Installing dependencies from Pipfile.lock (2110f5)...
All dependencies are now up-to-date!
Upgrading scikit-learn==1.5.0, pandas==2.3.0 in  dependencies.
Building requirements...
Resolving dependencies....
✔ Success! Locking packages...
⠋ Locking packages...
Building requirements...
Resolving dependencies....
✔ Success! Locking packages...
⠙ Locking packages...
Building requirements...
Resolving dependencies....
✔ Succe